# f) Ngonye Falls Power and Energy Periods
Produces summary, agrregated results from the daily power and energy modelling.


## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|
| [Model Name]_pe_daily.csv | Notebook: e_power_energy | Daily power output  |


## Outputs
| File                           | Description                                 |
|--------------------------------|---------------------------------------------|
|    [Model Name]_pe_daily.csv ||
|    [Model Name]_pe_monthly.csv||
|    [Model Name]_pe_yearly.csv||
|    [Model Name]_pe_percentiles.csv||
|    [Model Name]_pe_monthly_day.csv||
|    [Model Name]_pe_daily_slim.csv||
|    [Model Name]_pe_weekly.csv||
|    [Model Name]_pe_weekly_slim.csv||
|    [Model Name]_pe_calmonthly.csv||
|    [Model Name]_pe_fdc.csv||
|    [Model Name]_pe_power_exceed.csv||
|    models.csv| |

## Libraries and Setup

In [36]:
import numpy as np
import pandas as pd
from scipy import interpolate
import math
import statsmodels.tsa.stattools as statools

## Parameters

In [37]:
input_data='./input_data/'
#output_data='./output_data/2020/'
#output_data='./output_data/'
output_data='./output_data/2022/'

In [38]:
model_name='Base Case'
dryrun=False #Dont write output files

In [39]:
#models=pd.read_csv(input_data + "models.csv").set_index('ModelName')
#models=pd.read_csv(input_data + "2020/models.csv").set_index('ModelName')
models=pd.read_csv(input_data + "2022/models.csv").set_index('ModelName')
model=models.loc[model_name]

out_prefix=model['OutputPrefix']#'base'
if out_prefix!='base':
    out_prefix="/scenarios/" + out_prefix

category_set_name=model['EWRCatSet']#'Recommendation 1'
headpond_lift=model['HeadpondLift']#0
plant_capacity=model['Capacity'] #Base 180MW nominal capacity. Only used to calculate 'nominal capacity factor'
units=4 #Design no of units
units_avialable=model['UnitsAvailable'] #Available no of units - to see what happens during testing/commissioning

rated_flow_unit=model['RatedDischarge'] #250
max_flow_unit=rated_flow_unit*model['MaximumLoadFactor'] #275
canal_capacity=max_flow_unit*4 #canal capacity 1100 cumec 
min_flow_unit=model['MinimumFlowUnit'] #50cumec minimum flow for a turbine
fixed_tailwater_level=model['FixedTailwater']#False #970 
head_minimum=model['MinimumHead'] #minimum head 10m

rated_power_unit=rated_flow_unit*model['RatedHead']*model['RatedTurbineEfficiency']*model['RatedGeneratorEfficiency']*9.81/1000

model

OutputPrefix                            base
Description                        Base Case
EWRCatSet                   Recommendation 1
HeadpondLift                               0
Capacity                                 180
MinimumHead                               10
MinimumFlowUnit                           50
FixedTailwater                             0
UnitsAvailable                             4
RatedDischarge                           250
RatedHead                               21.7
RatedTurbineEfficiency                 0.943
RatedGeneratorEfficiency                0.98
MaximumLoadFactor                        1.1
RatedPower                            49.182
Mean                                  838165
P95                                   719611
P90                                   757122
P75                                   796138
P50                                   843034
P25                                   878919
P10                                   929634
P5        

## Load Daily Data

Load the daily time series which includes the exceedance values used for calculating daily EWRs.

In [51]:
daily=pd.read_csv(output_data + out_prefix + '_pe_daily.csv') #ngonye_daily_WC1_1_base_pe
daily=daily.set_index(pd.to_datetime(daily['Date'],format="%Y-%m-%d"))
daily=daily.drop('Date',axis=1)
daily

,LaggedDate,VicFalls,Conversion,Flow,Exceedance,Year,Month,Day,MonthId,WaterYear,...,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy,PerfMwPerCumec,PotentialPower1u,PotentialPower2u,PotentialPower3u,PotentialPower4u
Date,,,,,,,,,,,,,,,,,,,,,
1924-10-01,1924-10-12,100.0,1.384095,138.409487,0.998,1924,10,1,1924.10,1924,...,NaN,NaN,NaN,0.000000,0.000000,0.000000,57.577734,114.700482,170.813526,224.363600
1924-10-02,1924-10-13,100.0,1.384095,138.409487,0.998,1924,10,2,1924.10,1924,...,NaN,NaN,NaN,0.000000,0.000000,0.000000,57.577734,114.700482,170.813526,224.363600
1924-10-03,1924-10-14,100.0,1.384095,138.409487,0.998,1924,10,3,1924.10,1924,...,NaN,NaN,NaN,0.000000,0.000000,0.000000,57.577734,114.700482,170.813526,224.363600
1924-10-04,1924-10-15,100.0,1.384095,138.409487,0.998,1924,10,4,1924.10,1924,...,NaN,NaN,NaN,0.000000,0.000000,0.000000,57.577734,114.700482,170.813526,224.363600
1924-10-05,1924-10-16,100.0,1.384095,138.409487,0.998,1924,10,5,1924.10,1924,...,NaN,NaN,NaN,0.000000,0.000000,0.000000,57.577734,114.700482,170.813526,224.363600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-26,2022-10-07,293.0,0.932126,273.112795,0.825,2022,9,26,2022.09,2021,...,NaN,NaN,NaN,33.815564,811.573535,0.123815,57.203465,113.917905,169.554442,222.481697
2022-09-27,2022-10-08,289.0,0.928947,268.465689,0.837,2022,9,27,2022.09,2021,...,NaN,NaN,NaN,32.834283,788.022786,0.122303,57.218432,113.949138,169.604540,222.556235
2022-09-28,2022-10-09,286.0,0.929724,265.901164,0.847,2022,9,28,2022.09,2021,...,NaN,NaN,NaN,32.291455,774.994923,0.121442,57.226416,113.965800,169.631270,222.596015


In [52]:
#daily=daily.drop(daily.loc['2020-05-01':].index)

## Monthly


In [53]:

monthly=daily.groupby("MonthId").mean().drop(['Day','EWRBandNo','EWRRefExceedance','Energy'],axis=1)

if 'VicFalls' in monthly.columns:
    monthly=monthly.drop(['VicFalls','Conversion','Volume','WaterDay','WaterWeek','Volume'],axis=1)

monthly=monthly.join(daily.groupby("MonthId").agg(    
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),   
   Turbines_max=('Turbines', 'max'),    
   Turbines_min=('Turbines', 'min'),   
   Energy=('Energy', 'sum'),
   FlowAvailableForGeneration_min=('FlowAvailableForGeneration','min')
))
monthly['Days']=monthly.apply(lambda x: pd.Period(year=x['Year'],month=x['Month'],freq='D').days_in_month,axis=1)
monthly['CapFactor']=monthly['Energy']/(plant_capacity*24*monthly['Days'])

monthly

,Flow,Exceedance,Year,Month,WaterYear,WaterMonth,Flow_difference,Flow_difference_pct,EWRChannelA,EWRChannelC,...,PotentialPower3u,PotentialPower4u,Power_max,Power_min,Turbines_max,Turbines_min,Energy,FlowAvailableForGeneration_min,Days,CapFactor
MonthId,,,,,,,,,,,,,,,,,,,,,
1924.10,136.771526,0.998452,1924.0,10.0,1924.0,1.0,0.610130,0.020579,28.193548,4.322581,...,170.810614,224.359226,18.828159,0.000000,1,0,8767.783860,41.409487,31,0.065470
1924.11,153.655969,0.992400,1924.0,11.0,1924.0,2.0,2.353660,0.014794,23.000000,4.000000,...,170.896337,224.488134,31.166339,17.167260,1,1,15906.415492,82.587158,30,0.122735
1924.12,276.157068,0.831323,1924.0,12.0,1924.0,3.0,9.160736,0.029641,26.000000,4.000000,...,169.474046,222.365311,81.571064,29.453711,2,1,35543.558652,134.122845,31,0.265409
1925.01,704.059970,0.459323,1925.0,1.0,1924.0,4.0,44.430930,0.055882,37.129032,5.677419,...,160.741286,209.716186,192.336429,73.315246,4,2,88999.965733,345.554562,31,0.664576
1925.02,1210.241015,0.279000,1925.0,2.0,1924.0,5.0,25.090361,0.020239,49.000000,8.000000,...,148.461539,192.334639,191.342471,177.468973,4,4,123652.229395,964.382002,28,1.022257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022.05,2785.549155,0.106677,2022.0,5.0,2021.0,8.0,29.266761,0.011143,61.774194,13.677419,...,113.696839,144.677930,159.345788,135.028431,4,4,106952.145391,1898.594927,31,0.798627
2022.06,1534.964803,0.236667,2022.0,6.0,2021.0,9.0,40.784479,0.027095,62.000000,12.000000,...,140.822431,181.777242,187.022901,141.532436,4,3,122343.190151,739.060566,30,0.944006
2022.07,668.630149,0.469226,2022.0,7.0,2021.0,10.0,16.433611,0.023411,58.000000,11.000000,...,161.630834,210.960235,143.053610,56.210717,3,1,66605.123994,259.618631,31,0.497350


## Yearly

In [54]:
yearly=daily.groupby("WaterYear").agg(
   Flow_max=('Flow', 'max'),
   Flow_min=('Flow', 'min'), 
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_min=('EWRProportion', 'min'),
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_min=('HeadTurbine1', 'min'),   
   PowerTurbine1_max=('PowerTurbine1', 'max'),    
   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_min=('LevelTailwater', 'min'),  
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),    
   Energy=('Energy', 'sum'),
)
yearly['CapFactor']=yearly['Energy']/(plant_capacity*24*365)


In [55]:
#yearly=yearly.drop(yearly.loc['2019':].index)
yearly.tail(5)

,Flow_max,Flow_min,EWRProportion_max,EWRProportion_min,HeadTurbine1_max,HeadTurbine1_min,PowerTurbine1_max,LevelTailwater_max,LevelTailwater_min,Power_max,Power_min,Energy,CapFactor
WaterYear,,,,,,,,,,,,,
2017,4855.789688,171.188037,0.555972,0.055607,25.390592,10.970641,60.585296,977.092457,964.401315,188.857727,20.659607,843033.897521,0.534649
2018,1103.622889,158.591340,0.523314,0.109287,25.437359,20.415804,60.407077,968.222504,964.401315,180.553039,18.010865,727306.550364,0.461255
2019,4709.551434,130.594892,0.447791,0.054678,25.375426,11.208360,60.407077,976.854738,964.401315,190.694690,16.152746,810320.245193,0.513902
2020,3732.969062,261.920014,0.473687,0.050978,24.928288,12.939665,59.106629,975.123433,964.783973,191.526399,33.706928,912457.307002,0.578677
2021,3116.982122,265.901164,0.534663,0.069840,24.939484,14.191525,58.401728,973.871573,964.793595,191.354971,32.291455,890721.351445,0.564892


In [56]:
yearly['Prop4Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==4,1,0))/365)
yearly['Prop3Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==3,1,0))/365)
yearly['Prop2Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==2,1,0))/365)
yearly['Prop1Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==1,1,0))/365)
yearly['Prop0Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==0,1,0))/365)
yearly['Days0Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==0,1,0)))

In [57]:
yearly['PropDays1uPotentialGtRatedPower']=daily[['WaterYear','PotentialPower1u']].where(lambda x: x['PotentialPower1u']>rated_power_unit).groupby('WaterYear').count()
yearly['PropDays2uPotentialGtRatedPower']=daily[['WaterYear','PotentialPower2u']].where(lambda x: x['PotentialPower2u']>rated_power_unit*2).groupby('WaterYear').count()
yearly['PropDays3uPotentialGtRatedPower']=daily[['WaterYear','PotentialPower3u']].where(lambda x: x['PotentialPower3u']>rated_power_unit*3).groupby('WaterYear').count()
yearly['PropDays4uPotentialGtRatedPower']=daily[['WaterYear','PotentialPower4u']].where(lambda x: x['PotentialPower4u']>yearly['Power_max'].max()).groupby('WaterYear').count()

In [58]:
daily.reset_index()[['Date','WaterYear','PotentialPower1u']].where(lambda x: x['PotentialPower1u']<rated_power_unit).groupby('WaterYear').min()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)

WaterYear
1924.0     57
1925.0     66
1926.0     53
1927.0     67
1928.0    104
         ... 
2016.0     54
2017.0     40
2019.0     63
2020.0     37
2021.0     66
Name: Date, Length: 89, dtype: int64

In [59]:
yearly['FstDay1uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower1u']].where(lambda x: x['PotentialPower1u']<rated_power_unit).groupby('WaterYear').min()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)
yearly['LstDay1uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower1u']].where(lambda x: x['PotentialPower1u']<rated_power_unit).groupby('WaterYear').max()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)

yearly['FstDay2uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower2u']].where(lambda x: x['PotentialPower2u']<rated_power_unit*2).groupby('WaterYear').min()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)
yearly['LstDay2uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower2u']].where(lambda x: x['PotentialPower2u']<rated_power_unit*2).groupby('WaterYear').max()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)

yearly['FstDay3uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower3u']].where(lambda x: x['PotentialPower3u']<rated_power_unit*3).groupby('WaterYear').min()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)
yearly['LstDay3uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower3u']].where(lambda x: x['PotentialPower3u']<rated_power_unit*3).groupby('WaterYear').max()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)

yearly['FstDay4uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower4u']].where(lambda x: x['PotentialPower4u']<yearly['Power_max'].max()).groupby('WaterYear').min()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)
yearly['LstDay4uPotentialLessRatedPower']=daily.reset_index()[['Date','WaterYear','PotentialPower4u']].where(lambda x: x['PotentialPower4u']<yearly['Power_max'].max()).groupby('WaterYear').max()['Date'].apply(lambda x: pd.Timestamp(x).dayofyear)


## Power and energy exceedance

In [60]:
power_exceed=pd.DataFrame(index=np.arange(0,1.05,0.05))
power_exceed.index=power_exceed.index.rename('Exceedance')
power_exceed['Power']=np.percentile(daily['Power'],(1-power_exceed.index)*100)
power_exceed.head(3)

,Power
Exceedance,
0.00,194.044880
0.05,182.457700
0.10,173.944683


In [61]:
edc=pd.DataFrame(index=np.arange(0,1.05,0.05))
edc['Energy']=np.percentile(yearly['Energy'],(1-edc.index)*100)

edc.head(3)

,Energy
0.00,1.027124e+06
0.05,9.646171e+05
0.10,9.284781e+05


In [62]:
yearly['Exceedance']=pd.merge_asof(yearly.reset_index().sort_values('Energy'),edc.reset_index().sort_values('Energy'),left_on='Energy',right_on='Energy').set_index('WaterYear')['index']

## Calendar years
Jan-Dec rather than Oct-Sept

In [63]:
min_year=daily['Year'].min()
max_year=daily['Year'].max()


In [64]:
calyearly=daily.groupby("Year").agg(  
   Energy=('Energy', 'sum'),
).drop([min_year,max_year],axis=0)
calyearly['CapFactor']=calyearly['Energy']/(plant_capacity*24*365)
calyearly.head(5)

,Energy,CapFactor
Year,,
1925,859699.914800,0.545218
1926,784396.018845,0.497461
1927,858896.868857,0.544709
1928,835999.989452,0.530188
1929,763337.011596,0.484105


## Monthly Day
15th of every month

In [65]:
monthly_day=daily.loc[daily['Day']==15]

## Daily Slim
Slim has less columns and values are rounded so file size is reduced

In [66]:
daily_slim=daily.loc[:,
                     ['Year',
                      'Month',
                      'Day',
                      'WaterYear',
                      'Flow',
                      'EWRRefExceedance',
                      'EWRBandNo',
                      'EWRChannelA',
                      'EWRChannelC',
                      'EWRChannelD',
                      'EWRChannelE',
                      'EWRChannelFG',
                      'EWRTotal',
                      'EWRProportion',
                      'FlowCanal',
                      'FlowSpill',
                      'SpillChannelA',
                      'SpillChannelC',
                      'SpillChannelD',
                      'SpillChannelE',
                      'SpillChannelFG',
                      'FlowChannelA',
                      'FlowChannelC',
                      'FlowChannelD',
                      'FlowChannelE',
                      'FlowChannelFG',
                      'FlowLeftChannel',
                      'FlowTurbine1',
                      'FlowTurbine2',
                      'FlowTurbine3',
                      'FlowTurbine4',
                      'LevelHeadpond',
                      'LevelTailwater',
                      'HeadlossLeftChannel',
                      'HeadlossCanal',
                      'LevelForebay',
                      'Turbines',
                      'HeadTurbine1',
                      'HeadTurbine2',
                      'HeadTurbine3',
                      'HeadTurbine4',
                      'EffTurbine1',
                      'EffTurbine2',
                      'EffTurbine3',
                      'EffTurbine4',
                      'PowerTurbine1',
                      'PowerTurbine2',
                      'PowerTurbine3',
                      'PowerTurbine4',
                      'Power',
                      'Energy'
                     ]
                    ].fillna(0).astype({
                        'Flow': 'int32',
                        'FlowCanal': 'int32',
                        'FlowSpill': 'int32',
                        'EWRChannelA': 'int32',
                        'EWRChannelC': 'int32',
                        'EWRChannelD': 'int32',    
                        'EWRChannelE': 'int32',
                        'EWRChannelFG': 'int32',
                        'EWRTotal': 'int32',    
                        'SpillChannelA': 'int32',
                        'SpillChannelC': 'int32',
                        'SpillChannelD': 'int32',    
                        'SpillChannelE': 'int32',
                        'SpillChannelFG': 'int32',
                        'FlowChannelA': 'int32',
                        'FlowChannelC': 'int32',
                        'FlowChannelD': 'int32',    
                        'FlowChannelE': 'int32',
                        'FlowChannelFG': 'int32',
                        'FlowLeftChannel': 'int32',
                        'FlowTurbine1': 'int32',
                        'FlowTurbine2': 'int32',
                        'FlowTurbine3': 'int32',
                        'FlowTurbine4': 'int32',
                        'Energy': 'int32',
                        
                    }).round({
                            'EWRProportion': 2,
                            'LevelHeadpond': 1,
                            'LevelTailwater': 1,
                            'HeadlossLeftChannel': 2,
                            'HeadlossCanal': 2,
                            'LevelForebay': 1,
                            'PowerTurbine1': 1,
                            'PowerTurbine2': 1,
                            'PowerTurbine3': 1,
                            'PowerTurbine4': 1,
                            'Power': 1,
                            'HeadTurbine1':1,
                            'HeadTurbine2':1,
                            'HeadTurbine3':1,
                            'HeadTurbine4':1,
                            'EffTurbine1':2,
                            'EffTurbine2':2,
                            'EffTurbine3':2,
                            'EffTurbine4':2,    
                    })
daily_slim.tail(12)

,Year,Month,Day,WaterYear,Flow,EWRRefExceedance,EWRBandNo,EWRChannelA,EWRChannelC,EWRChannelD,...,EffTurbine1,EffTurbine2,EffTurbine3,EffTurbine4,PowerTurbine1,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy
Date,,,,,,,,,,,,,,,,,,,,,
2022-09-19,2022,9,19,2021,286,0.42,3,40,6,4,...,0.92,0.0,0.0,0.0,36.7,0.0,0.0,0.0,36.7,881
2022-09-20,2022,9,20,2021,283,0.42,3,40,6,4,...,0.92,0.0,0.0,0.0,36.1,0.0,0.0,0.0,36.1,866
2022-09-21,2022,9,21,2021,281,0.41,3,40,6,4,...,0.92,0.0,0.0,0.0,35.6,0.0,0.0,0.0,35.6,855
2022-09-22,2022,9,22,2021,282,0.41,3,40,6,4,...,0.92,0.0,0.0,0.0,35.7,0.0,0.0,0.0,35.7,856
2022-09-23,2022,9,23,2021,279,0.41,3,40,6,4,...,0.92,0.0,0.0,0.0,35.2,0.0,0.0,0.0,35.2,844
2022-09-24,2022,9,24,2021,274,0.41,3,40,6,4,...,0.92,0.0,0.0,0.0,34.2,0.0,0.0,0.0,34.2,820
2022-09-25,2022,9,25,2021,272,0.41,3,40,6,4,...,0.92,0.0,0.0,0.0,33.8,0.0,0.0,0.0,33.8,810
2022-09-26,2022,9,26,2021,273,0.41,3,40,6,4,...,0.92,0.0,0.0,0.0,33.8,0.0,0.0,0.0,33.8,811
2022-09-27,2022,9,27,2021,268,0.41,3,40,6,4,...,0.92,0.0,0.0,0.0,32.8,0.0,0.0,0.0,32.8,788


## Weekly & Weekly Slim
Slim has less columns and values are rounded so file size is reduced

In [67]:
weekly=daily.groupby(["WaterYear","WaterWeek"]).mean().drop(['Month','WaterMonth','Year','MonthId','Day','WaterDay','EWRBandNo','EWRRefExceedance','Turbines','Energy'],axis=1)

#if 'VicFalls' in weekly.columns:
weekly=weekly.drop(['VicFalls','Conversion','Volume'],axis=1)

weekly=weekly.join(daily.groupby(["WaterYear","WaterWeek"]).agg(    
   Flow_max=('Flow', 'max'),
   Flow_min=('Flow', 'min'), 
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_min=('EWRProportion', 'min'),
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_min=('HeadTurbine1', 'min'),    
   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_min=('LevelTailwater', 'min'),  
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),    
   Energy=('Energy', 'sum'),
))
weekly=weekly.join(daily[["WaterYear","WaterWeek","Flow"]].groupby(["WaterYear","WaterWeek"]).count().rename(columns={'Flow':'Count'}))
weekly['CapFactor']=weekly['Energy']/(plant_capacity*24*weekly['Count'])
weekly=weekly.join(daily.reset_index('Date')[["WaterYear","WaterWeek","Date"]].groupby(["WaterYear","WaterWeek"]).first().rename(columns={'Date':'StartDate'}))

weekly.tail(12)

Flow  Exceedance  Flow_difference  \
WaterYear WaterWeek                                            
2021      42         604.065639    0.490143        13.445609   
          43         534.657921    0.535143         8.809745   
          44         483.056208    0.571286         5.767399   
          45         442.671726    0.608286         6.118159   
          46         410.196894    0.642143         3.636412   
          47         380.962525    0.678000         4.358470   
          48         352.031809    0.710714         3.807089   
          49         329.501263    0.742429         2.972353   
          50         309.458675    0.770286         2.746306   
          51         288.344242    0.803714         2.922429   
          52         271.680619    0.832000         2.467244   
          53         266.795709    0.844000         0.000000   

                     Flow_difference_pct  EWRChannelA  EWRChannelC  \
WaterYear WaterWeek                                                  
2021      42                    0.022107    58.000000    11.000000   
          43                    0.016522    58.000000    11.000000   
          44                    0.011917    55.714286     9.857143   
          45                    0.013841    54.000000     9.000000   
          46                    0.008876    54.000000     9.000000   
          47                    0.011451    54.000000     9.000000   
          48                    0.010790    52.000000     8.571429   
          49                    0.009019    40.000000     6.000000   
          50                    0.008860    40.000000     6.000000   
          51                    0.010093    40.000000     6.000000   
          52                    0.009078    40.000000     6.000000   
          53                    0.000000    40.000000     6.000000   

                     EWRChannelD  EWRChannelE  EWRChannelFG    EWRTotal  ...  \
WaterYear WaterWeek                                                      ...   
2021      42           11.000000    11.000000    136.000000  227.000000  ...   
          43           11.000000    11.000000    136.000000  227.000000  ...   
          44            9.285714     8.142857    119.428571  202.428571  ...   
          45            8.000000     6.000000    107.000000  184.000000  ...   
          46            8.000000     6.000000    107.000000  184.000000  ...   
          47            8.000000     6.000000    107.000000  184.000000  ...   
          48            7.428571     5.571429    101.000000  174.571429  ...   
          49            4.000000     3.000000     65.000000  118.000000  ...   
          50            4.000000     3.000000     65.000000  118.000000  ...   
          51            4.000000     3.000000     65.000000  118.000000  ...   
          52            4.000000     3.000000     65.000000  118.000000  ...   
          53            4.000000     3.000000     65.000000  118.000000  ...   

                     HeadTurbine1_max  HeadTurbine1_min  LevelTailwater_max  \
WaterYear WaterWeek                                                           
2021      42                23.563747         23.064933          966.357805   
          43                23.938770         23.626680          965.962569   
          44                24.115671         23.612557          965.709008   
          45                24.162290         23.821471          965.521329   
          46                24.258753         24.081828          965.357223   
          47                24.483733         24.300697          965.240898   
          48                24.648950         24.460246          965.126849   
          49                24.609439         24.480432          965.026689   
          50                24.735294         24.633544          964.949352   
          51                24.855152         24.748829          964.890088   
          52                24.939484         24.867103          964.829773   
 

In [68]:
weekly_slim=weekly.loc[:,
                     [
                      'StartDate',
                      'Flow',
                      'EWRChannelA',
                      'EWRChannelC',
                      'EWRChannelD',
                      'EWRChannelE',
                      'EWRChannelFG',
                      'EWRTotal',
                      'EWRProportion',
                      'FlowCanal',
                      'FlowSpill',
                      'SpillChannelA',
                      'SpillChannelC',
                      'SpillChannelD',
                      'SpillChannelE',
                      'SpillChannelFG',
                      'FlowChannelA',
                      'FlowChannelC',
                      'FlowChannelD',
                      'FlowChannelE',
                      'FlowChannelFG',
                      'FlowLeftChannel',
                      'FlowTurbine1',
                      'FlowTurbine2',
                      'FlowTurbine3',
                      'FlowTurbine4',
                      'LevelHeadpond',
                      'LevelTailwater',
                      'HeadlossLeftChannel',
                      'HeadlossCanal',
                      'LevelForebay',
                      'HeadTurbine1',
                      'HeadTurbine2',
                      'HeadTurbine3',
                      'HeadTurbine4',
                      'EffTurbine1',
                      'EffTurbine2',
                      'EffTurbine3',
                      'EffTurbine4',
                      'PowerTurbine1',
                      'PowerTurbine2',
                      'PowerTurbine3',
                      'PowerTurbine4',
                      'Power',
                      'Energy'
                     ]
                    ].fillna(0).astype({
                        'Flow': 'int32',
                        'FlowCanal': 'int32',
                        'FlowSpill': 'int32',
                        'EWRChannelA': 'int32',
                        'EWRChannelC': 'int32',
                        'EWRChannelD': 'int32',    
                        'EWRChannelE': 'int32',
                        'EWRChannelFG': 'int32',
                        'EWRTotal': 'int32',
                        'SpillChannelA': 'int32',
                        'SpillChannelC': 'int32',
                        'SpillChannelD': 'int32',    
                        'SpillChannelE': 'int32',
                        'SpillChannelFG': 'int32',
                        'FlowChannelA': 'int32',
                        'FlowChannelC': 'int32',
                        'FlowChannelD': 'int32',    
                        'FlowChannelE': 'int32',
                        'FlowChannelFG': 'int32',
                        'FlowLeftChannel': 'int32',
                        'FlowTurbine1': 'int32',
                        'FlowTurbine2': 'int32',
                        'FlowTurbine3': 'int32',
                        'FlowTurbine4': 'int32',
                        'Energy': 'int32',
                        
                    }).round({
                            'EWRProportion':2,
                            'LevelHeadpond': 1,
                            'LevelTailwater': 1,
                            'HeadlossLeftChannel': 2,
                            'HeadlossCanal': 2,
                            'LevelForebay': 1,
                            'PowerTurbine1': 1,
                            'PowerTurbine2': 1,
                            'PowerTurbine3': 1,
                            'PowerTurbine4': 1,
                            'Power': 1,
                            'HeadTurbine1':1,
                            'HeadTurbine2':1,
                            'HeadTurbine3':1,
                            'HeadTurbine4':1,
                            'EffTurbine1':2,
                            'EffTurbine2':2,
                            'EffTurbine3':2,
                            'EffTurbine4':2,
                    })

## Calendar Months

In [69]:
calmonthly=pd.DataFrame({'WaterMonth': [1,2,3,4,5,6,7,8,9,10,11,12],'MonthName': ['Oct','Nov','Dec','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep'], 'Month':[10,11,12,1,2,3,4,5,6,7,8,9]})
calmonthly=calmonthly.set_index('WaterMonth')
calmonthly=calmonthly.join(monthly.groupby(["WaterMonth"]).agg(    
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_mean=('EWRProportion', 'mean'),
   EWRProportion_median=('EWRProportion', 'median'),
   EWRProportion_min=('EWRProportion', 'min'),
    
    EWRTotal_max=('EWRTotal', 'max'),
   EWRTotal_mean=('EWRTotal', 'mean'),
   EWRTotal_median=('EWRTotal', 'median'),
   EWRTotal_min=('EWRTotal', 'min'),   
    
   FlowCanal_max=('FlowCanal', 'max'),
   FlowCanal_mean=('FlowCanal', 'mean'),
   FlowCanal_median=('FlowCanal', 'median'),
   FlowCanal_min=('FlowCanal', 'min'),     
    
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_mean=('HeadTurbine1', 'mean'),
   HeadTurbine1_median=('HeadTurbine1', 'median'),
   HeadTurbine1_min=('HeadTurbine1', 'min'), 

   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_mean=('LevelTailwater', 'mean'),    
   LevelTailwater_median=('LevelTailwater', 'median'),    
   LevelTailwater_min=('LevelTailwater', 'min'),  
   LevelHeadpond_max=('LevelHeadpond', 'max'),
   LevelHeadpond_mean=('LevelHeadpond', 'mean'),    
   LevelHeadpond_median=('LevelHeadpond', 'median'),    
   LevelHeadpond_min=('LevelHeadpond', 'min'),  
   Turbines_mean=('Turbines', 'mean'),      
   Power_max=('Power', 'max'),
   Power_mean=('Power', 'mean'),
   Power_median=('Power', 'median'),
   Power_min=('Power', 'min'),    
   Energy_max=('Energy', 'max'),
   Energy_mean=('Energy', 'mean'),
   Energy_median=('Energy', 'median'),    
   Energy_min=('Energy', 'min'),
))
calmonthly['Energy_P95']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.05)
calmonthly['Energy_P90']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.10)
calmonthly['Energy_P75']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.25)
calmonthly['Energy_P25']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.75)
calmonthly['Energy_P10']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.90)
calmonthly['Energy_P5']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.95)

calmonthly['Power_P95']=daily[['WaterMonth','Power']].groupby('WaterMonth').quantile(0.05)
calmonthly['Power_P90']=daily[['WaterMonth','Power']].groupby('WaterMonth').quantile(0.10)

calmonthly['FlowAvailableForGeneration_P95']=monthly[['WaterMonth','FlowAvailableForGeneration']].groupby('WaterMonth').quantile(0.05)
calmonthly['FlowAvailableForGeneration_P50']=monthly[['WaterMonth','FlowAvailableForGeneration']].groupby('WaterMonth').quantile(0.5)

calmonthly['PotentialPower1u_P95']=monthly[['WaterMonth','PotentialPower1u']].groupby('WaterMonth').quantile(0.05)
calmonthly['PotentialPower1u_P50']=monthly[['WaterMonth','PotentialPower1u']].groupby('WaterMonth').quantile(0.5)
calmonthly['PotentialPower2u_P95']=monthly[['WaterMonth','PotentialPower2u']].groupby('WaterMonth').quantile(0.05)
calmonthly['PotentialPower2u_P50']=monthly[['WaterMonth','PotentialPower2u']].groupby('WaterMonth').quantile(0.5)
calmonthly['PotentialPower3u_P95']=monthly[['WaterMonth','PotentialPower3u']].groupby('WaterMonth').quantile(0.05)
calmonthly['PotentialPower3u_P50']=monthly[['WaterMonth','PotentialPower3u']].groupby('WaterMonth').quantile(0.5)
calmonthly['PotentialPower4u_P95']=monthly[['WaterMonth','PotentialPower4u']].groupby('WaterMonth').quantile(0.05)
calmonthly['PotentialPower4u_P50']=monthly[['WaterMonth','PotentialPower4u']].groupby('WaterMonth').quantile(0.5)


calmonthly['Energy_stdev']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').std()
calmonthly['Energy_coefvar']=calmonthly['Energy_stdev']/calmonthly['Energy_mean']

calmonthly

,MonthName,Month,EWRProportion_max,EWRProportion_mean,EWRProportion_median,EWRProportion_min,EWRTotal_max,EWRTotal_mean,EWRTotal_median,EWRTotal_min,...,PotentialPower1u_P95,PotentialPower1u_P50,PotentialPower2u_P95,PotentialPower2u_P50,PotentialPower3u_P95,PotentialPower3u_P50,PotentialPower4u_P95,PotentialPower4u_P50,Energy_stdev,Energy_coefvar
WaterMonth,,,,,,,,,,,,,,,,,,,,,
1,Oct,10,0.485351,0.393410,0.397798,0.272949,120.0,93.492100,97.000000,52.0,...,56.915013,57.308511,113.316921,114.137218,168.592708,169.906490,221.055898,223.006083,6785.508187,0.287881
2,Nov,11,0.400612,0.332139,0.336252,0.214463,114.0,89.425170,93.000000,50.0,...,56.539449,57.215237,112.536899,113.942521,167.350409,169.594050,219.227601,222.540907,8934.346269,0.306332
3,Dec,12,0.287921,0.241665,0.243008,0.153900,126.0,100.154378,104.000000,59.0,...,54.567029,56.569237,108.471131,112.599471,160.949874,167.451772,209.978053,219.380664,14920.534504,0.284147
4,Jan,1,0.222306,0.174572,0.178459,0.097430,150.0,117.683015,120.000000,73.0,...,50.029792,55.012172,99.225570,109.384167,146.657153,162.376115,189.927738,212.013570,20445.242285,0.239636
5,Feb,2,0.186837,0.127101,0.136113,0.023905,197.0,138.675304,137.000000,85.0,...,37.054608,52.241798,73.023029,103.713538,106.739139,153.546909,135.318323,199.479558,18266.297187,0.183778
6,Mar,3,0.179385,0.094140,0.090588,0.040404,315.0,193.714286,192.032258,107.0,...,24.215843,42.247860,47.340173,83.468013,56.900099,122.544898,31.680993,156.685057,30237.381889,0.291243
7,Apr,4,0.160675,0.084586,0.080082,0.050051,364.0,231.173469,217.000000,125.0,...,26.729017,38.248311,52.337372,75.416712,75.644098,110.343230,84.341295,140.146292,26784.827174,0.273782
8,May,5,0.218068,0.129111,0.126487,0.065455,340.0,264.831797,276.000000,147.0,...,33.587807,42.787078,66.071315,84.556223,96.272545,124.200970,121.298836,158.946474,16701.517216,0.147014
9,Jun,6,0.369599,0.195805,0.191487,0.108597,293.0,212.918367,210.000000,119.0,...,44.166522,50.730067,87.341024,100.638277,128.441235,148.805738,164.742176,192.858696,22888.945678,0.214965


In [70]:
calmonthly[['Power_P90','Power_P95']]

,Power_P90,Power_P95
WaterMonth,,
1,22.420876,20.274339
2,26.454204,23.744246
3,44.076766,39.813988
4,76.768693,70.077569
5,109.132315,97.057944
6,96.661410,0.000000
7,96.664314,87.102914
8,120.614589,108.106345
9,82.939192,65.145670


In [71]:
calmonthly['PropYearsMinAvailFlowGt1UnitRatedFlow']=monthly[['WaterMonth','FlowAvailableForGeneration_min']].where(lambda x: x['FlowAvailableForGeneration_min']>rated_flow_unit).groupby('WaterMonth').count()/len(yearly.index)
calmonthly['PropYearsMinAvailFlowGt2UnitRatedFlow']=monthly[['WaterMonth','FlowAvailableForGeneration_min']].where(lambda x: x['FlowAvailableForGeneration_min']>rated_flow_unit*2).groupby('WaterMonth').count()/len(yearly.index)
calmonthly['PropYearsMinAvailFlowGt3UnitRatedFlow']=monthly[['WaterMonth','FlowAvailableForGeneration_min']].where(lambda x: x['FlowAvailableForGeneration_min']>rated_flow_unit*3).groupby('WaterMonth').count()/len(yearly.index)
calmonthly['PropYearsMinAvailFlowGtCanalRatedFlow']=monthly[['WaterMonth','FlowAvailableForGeneration_min']].where(lambda x: x['FlowAvailableForGeneration_min']>canal_capacity).groupby('WaterMonth').count()/len(yearly.index)
calmonthly.fillna(0)
calmonthly

,MonthName,Month,EWRProportion_max,EWRProportion_mean,EWRProportion_median,EWRProportion_min,EWRTotal_max,EWRTotal_mean,EWRTotal_median,EWRTotal_min,...,PotentialPower3u_P95,PotentialPower3u_P50,PotentialPower4u_P95,PotentialPower4u_P50,Energy_stdev,Energy_coefvar,PropYearsMinAvailFlowGt1UnitRatedFlow,PropYearsMinAvailFlowGt2UnitRatedFlow,PropYearsMinAvailFlowGt3UnitRatedFlow,PropYearsMinAvailFlowGtCanalRatedFlow
WaterMonth,,,,,,,,,,,,,,,,,,,,,
1,Oct,10,0.485351,0.393410,0.397798,0.272949,120.0,93.492100,97.000000,52.0,...,168.592708,169.906490,221.055898,223.006083,6785.508187,0.287881,0.030612,NaN,NaN,NaN
2,Nov,11,0.400612,0.332139,0.336252,0.214463,114.0,89.425170,93.000000,50.0,...,167.350409,169.594050,219.227601,222.540907,8934.346269,0.306332,0.051020,NaN,NaN,NaN
3,Dec,12,0.287921,0.241665,0.243008,0.153900,126.0,100.154378,104.000000,59.0,...,160.949874,167.451772,209.978053,219.380664,14920.534504,0.284147,0.265306,0.020408,NaN,NaN
4,Jan,1,0.222306,0.174572,0.178459,0.097430,150.0,117.683015,120.000000,73.0,...,146.657153,162.376115,189.927738,212.013570,20445.242285,0.239636,0.969388,0.224490,0.051020,NaN
5,Feb,2,0.186837,0.127101,0.136113,0.023905,197.0,138.675304,137.000000,85.0,...,106.739139,153.546909,135.318323,199.479558,18266.297187,0.183778,1.000000,0.857143,0.377551,0.122449
6,Mar,3,0.179385,0.094140,0.090588,0.040404,315.0,193.714286,192.032258,107.0,...,56.900099,122.544898,31.680993,156.685057,30237.381889,0.291243,1.000000,0.979592,0.806122,0.551020
7,Apr,4,0.160675,0.084586,0.080082,0.050051,364.0,231.173469,217.000000,125.0,...,75.644098,110.343230,84.341295,140.146292,26784.827174,0.273782,1.000000,1.000000,0.959184,0.816327
8,May,5,0.218068,0.129111,0.126487,0.065455,340.0,264.831797,276.000000,147.0,...,96.272545,124.200970,121.298836,158.946474,16701.517216,0.147014,1.000000,0.948980,0.897959,0.734694
9,Jun,6,0.369599,0.195805,0.191487,0.108597,293.0,212.918367,210.000000,119.0,...,128.441235,148.805738,164.742176,192.858696,22888.945678,0.214965,0.948980,0.602041,0.275510,0.061224


In [72]:
calmonthly['PropYears1uPotentialGtRatedPower']=monthly[['WaterMonth','PotentialPower1u']].where(lambda x: x['PotentialPower1u']>rated_power_unit).groupby('WaterMonth').count()/len(yearly.index)
calmonthly['PropYears2uPotentialGtRatedPower']=monthly[['WaterMonth','PotentialPower2u']].where(lambda x: x['PotentialPower2u']>rated_power_unit*2).groupby('WaterMonth').count()/len(yearly.index)
calmonthly['PropYears3uPotentialGtRatedPower']=monthly[['WaterMonth','PotentialPower3u']].where(lambda x: x['PotentialPower3u']>rated_power_unit*3).groupby('WaterMonth').count()/len(yearly.index)
calmonthly['PropYears4uPotentialGtRatedPower']=monthly[['WaterMonth','PotentialPower4u']].where(lambda x: x['PotentialPower4u']>yearly['Power_max'].max()).groupby('WaterMonth').count()/len(yearly.index)

calmonthly.fillna(0)
calmonthly


,MonthName,Month,EWRProportion_max,EWRProportion_mean,EWRProportion_median,EWRProportion_min,EWRTotal_max,EWRTotal_mean,EWRTotal_median,EWRTotal_min,...,Energy_stdev,Energy_coefvar,PropYearsMinAvailFlowGt1UnitRatedFlow,PropYearsMinAvailFlowGt2UnitRatedFlow,PropYearsMinAvailFlowGt3UnitRatedFlow,PropYearsMinAvailFlowGtCanalRatedFlow,PropYears1uPotentialGtRatedPower,PropYears2uPotentialGtRatedPower,PropYears3uPotentialGtRatedPower,PropYears4uPotentialGtRatedPower
WaterMonth,,,,,,,,,,,,,,,,,,,,,
1,Oct,10,0.485351,0.393410,0.397798,0.272949,120.0,93.492100,97.000000,52.0,...,6785.508187,0.287881,0.030612,NaN,NaN,NaN,1.000000,1.000000,1.000000,1.000000
2,Nov,11,0.400612,0.332139,0.336252,0.214463,114.0,89.425170,93.000000,50.0,...,8934.346269,0.306332,0.051020,NaN,NaN,NaN,1.000000,1.000000,1.000000,1.000000
3,Dec,12,0.287921,0.241665,0.243008,0.153900,126.0,100.154378,104.000000,59.0,...,14920.534504,0.284147,0.265306,0.020408,NaN,NaN,1.000000,1.000000,1.000000,1.000000
4,Jan,1,0.222306,0.174572,0.178459,0.097430,150.0,117.683015,120.000000,73.0,...,20445.242285,0.239636,0.969388,0.224490,0.051020,NaN,0.959184,0.948980,0.938776,0.908163
5,Feb,2,0.186837,0.127101,0.136113,0.023905,197.0,138.675304,137.000000,85.0,...,18266.297187,0.183778,1.000000,0.857143,0.377551,0.122449,0.734694,0.724490,0.693878,0.642857
6,Mar,3,0.179385,0.094140,0.090588,0.040404,315.0,193.714286,192.032258,107.0,...,30237.381889,0.291243,1.000000,0.979592,0.806122,0.551020,0.306122,0.295918,0.275510,0.224490
7,Apr,4,0.160675,0.084586,0.080082,0.050051,364.0,231.173469,217.000000,125.0,...,26784.827174,0.273782,1.000000,1.000000,0.959184,0.816327,0.153061,0.132653,0.091837,0.071429
8,May,5,0.218068,0.129111,0.126487,0.065455,340.0,264.831797,276.000000,147.0,...,16701.517216,0.147014,1.000000,0.948980,0.897959,0.734694,0.163265,0.163265,0.132653,0.122449
9,Jun,6,0.369599,0.195805,0.191487,0.108597,293.0,212.918367,210.000000,119.0,...,22888.945678,0.214965,0.948980,0.602041,0.275510,0.061224,0.653061,0.622449,0.540816,0.459184


## Flow duration curve

In [73]:
daily['ExceedanceRnd']=np.round(daily['Exceedance']*2,2)/2
fdc=daily.reset_index().groupby('ExceedanceRnd').mean().drop(['VicFalls','Conversion','Year','Month','Day','MonthId','WaterYear','WaterDay','WaterMonth','WaterWeek'],axis=1)
fdc

,Flow,Exceedance,Volume,Flow_difference,Flow_difference_pct,EWRRefExceedance,EWRBandNo,EWRChannelA,EWRChannelC,EWRChannelD,...,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy,PerfMwPerCumec,PotentialPower1u,PotentialPower2u,PotentialPower3u,PotentialPower4u
ExceedanceRnd,,,,,,,,,,,,,,,,,,,,,
0.000,8809.879593,0.001431,0.761174,141.407689,0.016459,0.026667,1.097222,66.083333,12.430556,14.861111,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.005,6389.293248,0.004989,0.552035,109.134863,0.016749,0.043611,1.100000,72.427778,13.838889,17.688889,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.506935,22.466286,18.795424,0.000000
0.010,5563.531659,0.009983,0.480689,65.252467,0.011744,0.083204,1.320442,72.668508,13.574586,17.591160,...,8.685289,8.685289,8.685289,34.741155,833.787711,0.006379,24.862055,48.608370,70.067072,44.425684
0.015,5203.203065,0.014944,0.449557,89.050316,0.017197,0.101611,1.388889,71.366667,12.933333,16.600000,...,22.836167,22.836167,22.836167,91.344667,2192.271998,0.017571,26.352158,51.582774,74.510725,92.278791
0.020,4839.769106,0.020054,0.418156,93.641639,0.019328,0.110598,1.402174,69.554348,13.195652,16.597826,...,24.421997,24.421997,24.421997,97.687988,2344.511712,0.020200,27.958747,54.792348,79.312530,98.635084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.980,181.295707,0.980401,0.015664,2.697233,0.014865,0.893882,4.447368,28.546053,4.355263,2.782895,...,NaN,NaN,NaN,24.774158,594.579785,0.136628,57.650215,114.852454,171.059042,224.732834
0.985,172.231000,0.985087,0.014881,1.883828,0.010890,0.910510,4.525510,27.591837,4.229592,2.571429,...,NaN,NaN,NaN,23.384048,561.217146,0.135765,57.630073,114.810207,170.990755,224.630056
0.990,163.652909,0.990268,0.014140,1.881114,0.011482,0.944524,4.857143,25.636905,4.333333,2.392857,...,NaN,NaN,NaN,22.704541,544.908990,0.138674,57.613009,114.774424,170.932938,224.543087


## Add overall summary data to the model file

In [74]:

models.at[model_name,'Mean']=yearly['Energy'].mean()
models.at[model_name,'Max']=yearly['Energy'].max()
models.at[model_name,'Min']=yearly['Energy'].min()

models.at[model_name,'P95']=yearly['Energy'].quantile(0.05)
models.at[model_name,'P90']=yearly['Energy'].quantile(0.10)
models.at[model_name,'P75']=yearly['Energy'].quantile(0.25)
models.at[model_name,'P50']=yearly['Energy'].quantile(0.5)
models.at[model_name,'P25']=yearly['Energy'].quantile(0.75)
models.at[model_name,'P10']=yearly['Energy'].quantile(0.90)
models.at[model_name,'P5']=yearly['Energy'].quantile(0.95)

models.at[model_name,'StDev']=yearly['Energy'].std()
models.at[model_name,'CoefVar']=models.at[model_name,'StDev']/models.at[model_name,'Mean']

models.at[model_name,'CapFactor']=yearly['CapFactor'].mean()

models.at[model_name,'CalYearMean']=calyearly['Energy'].mean()
models.at[model_name,'CalYearStDev']=calyearly['Energy'].std()
models.at[model_name,'CalYearCoefVar']=models.at[model_name,'CalYearStDev']/models.at[model_name,'CalYearMean']

models.at[model_name,'Power_max']=yearly['Power_max'].max()
models.at[model_name,'Power_Turbine1_max']=yearly['PowerTurbine1_max'].max()

models.at[model_name,'CapFactor_Power_max']=calyearly['Energy'].mean()/(yearly['Power_max'].max()*24*365)

models.at[model_name,'RatedPower']=rated_power_unit

models.loc[model_name]



OutputPrefix                            base
Description                        Base Case
EWRCatSet                   Recommendation 1
HeadpondLift                               0
Capacity                                 180
MinimumHead                               10
MinimumFlowUnit                           50
FixedTailwater                             0
UnitsAvailable                             4
RatedDischarge                           250
RatedHead                               21.7
RatedTurbineEfficiency                 0.943
RatedGeneratorEfficiency                0.98
MaximumLoadFactor                        1.1
RatedPower                            49.182
Mean                                  839175
P95                                   723459
P90                                   758220
P75                                   798945
P50                                   843472
P25                                   881628
P10                                   928478
P5        

In [75]:
autocorrel=pd.DataFrame(statools.acf(yearly['Energy'],fft=True,qstat=False,alpha=0.05))
autocorrel

c:\Users\Andy Fleming\AppData\Local\Programs\Python\Python38\lib\site-packages\statsmodels\tsa\stattools.py:657: FutureWarning: The default number of lags is changing from 40 tomin(int(10 * np.log10(nobs)), nobs - 1) after 0.12is released. Set the number of lags to an integer to  silence this warning.
  warnings.warn(


,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,1,0.470097,0.320801,0.268956,0.250304,0.154117,0.137246,0.102364,0.172522,0.10544,...,-0.183813,-0.0921876,0.00497975,0.0850195,0.0356735,-0.026099,-0.0036224,0.0113216,-0.0292558,0.0235059
1,"[1.0, 1.0]","[0.2721105738432667, 0.6680830950860317]","[0.08305420079547007, 0.5585481322858595]","[0.014807152164150239, 0.5231052255845063]","[-0.014767551460478201, 0.5153749919129325]","[-0.12006305723507543, 0.42829635638014735]","[-0.14030855389214272, 0.41480078819507005]","[-0.17783815407302395, 0.38256642294394183]","[-0.10914254678797192, 0.45418613208693986]","[-0.1803362700792678, 0.3912166805381776]",...,"[-0.5448032628505778, 0.17717687775755453]","[-0.45682809426662907, 0.27245280367756153]","[-0.36057315082048846, 0.3705326412422224]","[-0.280536100596255, 0.45057500964041075]","[-0.33065636046839203, 0.40200329413568364]","[-0.3925649586178945, 0.34036698979047025]","[-0.37016122697768844, 0.3629164251020301]","[-0.3552185891919738, 0.3778618694347563]","[-0.39580969834916385, 0.337298175326543]","[-0.3431395453317228, 0.39015136194178285]"


In [76]:
lowheadshutdowns=daily.loc[daily['LowHeadShutoff']==1].groupby('WaterYear').count()['Flow'].to_frame().rename(columns={"Flow":"Days"})

lowheadshutdowns['StartDate']=daily[['WaterYear','Flow']].loc[daily['LowHeadShutoff']==1].reset_index().groupby('WaterYear').min()['Date']
lowheadshutdowns['EndDate']=daily[['WaterYear','Flow']].loc[daily['LowHeadShutoff']==1].reset_index().groupby('WaterYear').max()['Date']
lowheadshutdowns['MonthId']=daily[['WaterYear','MonthId']].loc[daily['LowHeadShutoff']==1].reset_index().groupby('WaterYear').min()['MonthId']

#daily.loc[daily['LowHeadShutoff']==1].groupby('WaterYear').min()#['Date']
lowheadshutdowns['Month_power_max']=monthly.loc[lowheadshutdowns['MonthId']][['WaterYear','Power_max']].set_index('WaterYear')
lowheadshutdowns

,Days,StartDate,EndDate,MonthId,Month_power_max
WaterYear,,,,,
1933,4,1934-03-25,1934-03-28,1934.03,181.661290
1947,32,1948-03-04,1948-04-04,1948.03,104.685546
1951,20,1952-03-02,1952-03-21,1952.03,103.852197
1952,10,1953-03-21,1953-03-30,1953.03,188.147774
1955,13,1956-05-05,1956-05-17,1956.05,114.089472
1956,27,1957-03-02,1957-03-28,1957.03,104.346731
1957,60,1958-02-03,1958-04-03,1958.02,108.221746
1960,22,1961-04-02,1961-04-23,1961.04,98.033675
1961,11,1962-04-04,1962-04-14,1962.04,102.107043


In [77]:
lowflowshutdowns=daily.loc[daily['LowFlowShutoff']==1].groupby('Year').count()['Flow'].to_frame().rename(columns={"Flow":"Days"})

lowflowshutdowns['StartDate']=daily[['Year','Flow']].loc[daily['LowFlowShutoff']==1].reset_index().groupby('Year').min()['Date']
lowflowshutdowns['EndDate']=daily[['Year','Flow']].loc[daily['LowFlowShutoff']==1].reset_index().groupby('Year').max()['Date']
lowflowshutdowns['MonthId']=daily[['Year','MonthId']].loc[daily['LowFlowShutoff']==1].reset_index().groupby('Year').min()['MonthId']


lowflowshutdowns['Month_power_max']=monthly.loc[lowflowshutdowns['MonthId']][['Year','Power_max']].set_index('Year')
lowflowshutdowns

,Days,StartDate,EndDate,MonthId,Month_power_max
Year,,,,,
1924,10,1924-10-01,1924-10-10,1924.1,18.828159


## Save Files

In [78]:
outp=output_data+out_prefix
#daily.to_csv(outp+'_pe_daily.csv')
calmonthly.to_csv(outp+'_pe_calmonthly.csv')

In [79]:
outp=output_data+out_prefix

if dryrun==False:
    daily.to_csv(outp+'_pe_daily.csv')
    monthly.to_csv(outp+'_pe_monthly.csv')
    yearly.to_csv(outp+'_pe_yearly.csv')
    edc.to_csv(outp+'_pe_percentiles.csv')
    monthly_day.to_csv(outp+'_pe_monthly_day.csv')
    daily_slim.to_csv(outp+'_pe_daily_slim.csv')
    weekly.to_csv(outp+'_pe_weekly.csv')
    weekly_slim.to_csv(outp+'_pe_weekly_slim.csv')
    calmonthly.to_csv(outp+'_pe_calmonthly.csv')
    power_exceed.to_csv(outp+'_pe_power_exceed.csv')
    fdc.to_csv(outp+'_pe_fdc.csv')
    autocorrel.to_csv(outp+'_pe_autocorrel.csv')
    #models.to_csv(input_data + "models.csv")
    models.to_csv(input_data + "2022/models.csv")
    lowheadshutdowns.to_csv(outp + "lowheadshutdowns.csv")
    lowflowshutdowns.to_csv(outp + "lowflowshutdowns.csv")

In [80]:
years=daily['WaterYear'].unique().tolist()

for year in years:
    days=daily.loc[daily.WaterYear==year]
    #days.to_csv(output_data + 'years/' + out_prefix + '_pe_daily_' + str(year) + '.csv')
 